# Chapter 1 - Notes To The Reader

# Chapter 2 - A Tour Of C++: The Basics

# Chapter 3 - A Tour Of C++: Abstraction Mechanisms

# Chapter 4 - A Tour Of C++: Containers And Algorithms

# Chapter 5 - A Tour Of C++: Concurrency And Utilities

# Chapter 6 - Types And Declarations

# Chapter 7 - Pointers, Arrays, And References


# Chapter 8 - Structures Unions, And Enumerations

### Section 8.4.2 - Plain Enums

Enumerators, i.e., members of <b>plain enum</b> have their scope "leaked" to the same scope as of their enum. They are also implicitly converted to int, i.e., their value can be used in int's, even though the converse is not possible. E.g.:

In [1]:
#include<iostream>

enum Leaking{ 
    
    first_leak, second_leak
};

int get_value = second_leak; // example of scope leak

std::cout << get_value << "\n";

int receiver = Leaking::first_leak; // Example of implicit conversion from enum to int

std::cout << receiver << "\n";

// Leaking leak = 4 // This case is not possible.

1
0


@0x7f6942a24b60

Because of this leakage problem, called <b>Namespace Pollution</b>, avoid common names for enumerators of <b>plain enum</b>. Try to use names that provide some info about the fact that the enumerator is indeed a member of a specific enum type. Disambiguation is the core idea behind this.

As for enum class, it is also possible to specify the <i>underlying integral type</i> of a <b>plain enum</b>; E.g.:

In [2]:
// Example of underlyint integral type specification
enum underlying_type : char{
    
    first, second, third
};

enum forward_declared : int; // Example of forward declaration of plain enum

This also make it possible for we to <b>forward declare</b> an enum, just like in the last line above. Without specifying an underlyint type, forward declarations are not allowed.

We said above that implicit conversions from int to plain enum is not possible, but this is not the case for <b>explicit</b> conversions. But, be careful, this is only <b>DEFINED</b> when the value of the int that will be used in the conversion to a plain enum is <b>within the range</b> of the enumeration. This range is defined as the smallest bit-field capable of holding the enumerators, i.e., members, values( almost a literal quote from Stroustrup ). This bit-field is calculated from the smallest k s.t. $2^k$( or $-2^k$ ) covers the range of values of the enumerators. If a value is <b>OUTSIDE</b> this range, then we have the famous <b>Undefined Behavior</b>.

A last note about conversions: since plain enum has an underlying integral type, there is indeed an <b>implicit conversion</b> from the former to the latter. This also means that

In [3]:
enum example{
    
    just_a_value
};

sizeof( example )

4

<b>enum example</b> has the <b>sizeof</b> its underlying, else explicitly specified, type. If the type is not explicityly specified, then the type will be of some integral type that can hold the range of values provided, where this type is no larger than <b>sizeof( int )</b>.

# Chapter 9 - Statements

## Section 9.1 - Introduction

Statements do not have values. Instead, they are used to specify the order of execution of the program.

## Section 9.2 - Statement Summary

There is a list of what are considered statements on page 226.

<b>Declarations</b> are statements whenever no initialization or procedure-calls are done. If one of the latter two happens, then we have an expression.

Sequences of statements made within curly braces are considered <b>block of code</b> or <b>compound statements</b>.

## Section 9.3 - Declarations As Statements

As said above, declarations are statements, and the initialization of a declared entity happens when the thread of control reaches the declared entity.

Consider defining user-defined types only when a suitable value is available, so as to initialize at construction. This takes less instructions to be performed, leading to better performance. Only consider initialization <i>After</i> declaration when it is not possible to do as previously stated, i.e., when the value for the entity depends on another statement. E.g.: when using

In [ ]:
#include<iostream>

int value{};

std::cin >> value;

std::cout << value << "\n";

where <b>int value</b> value depends on the input of <b>std::cin</b>.

## Section 9.4 - Selection Statements

Related to <b>if( ), if( ) else, switch( )</b> and <b>else if( )</b> statements.

Conditions are either expressions or declarations.

### Section 9.4.1 - <span style="color: blue;">if</span> statements

Evaluates a condition and executes an or many instructions, statements, if the condition come out to be true. If the latter is not true, an <b>else</b> statement, if provided, will execute its own set of instructions.

Arguments used in conditions can be <b>implicitly converted</b> to booleans, e.g.: arithmetic expressions and pointer expressions. Yeah, I know, dangerous and shitty, but what can we do about it? Simple: avoid passing arguments that can  be implicitly converted!

<b>Note:</b> consider using expressions with types for which <b>explicit</b> constructros were provided. This avoids implicit conversions when dealing with user-defined types, and will possibly save you from some NASTY trouble.

Consider the following snippets, they explain how implicit conversions works for certain cases:

In [ ]:
int x{ };

if( x ){ // This statemente is equivalent to
}

if( x != 0 ){ // this statement
}

int* pointer{ };

if( pointer ){ // This statement is equivalent to
}

if( pointer != nullptr ){ // This statement
}

enum implicit_conversion{
    
    condition_one, condition_two
};

implicit_conversion test;

if( test ){ // This statement is valid, because plain enums are implicitly convertible to int
}

enum class no_conversion{
    
    no_condition
};

no_conversion test_2;

/*if( test_2 ){ // This statement is not valid, because enum class is not implicitly convertible to int
}
*/

Binary logical operators like && and || only evaluates the second argument if necessary. I.e., they are <b>lazy</b>. An example would be an expression A && B, where A is false. It is not necessary for B to be evaluated at all, since the expression will be false whichever truth value B has.

<b>if statements'</b> block of code provides boundaries to entities declared in it. I.e., anything declared in an if-block cannot be used outside this block, not even in the else block associated to the if part. E.g.:

In [ ]:
if( true ){
    
    int value{ }; // value is bound to if scope. It cannot be accesses outside these brackets
    
    ++value;
}

else{
    
    // do something else
    
    // value--; // This is not possible, since value was not declared in here
}

But, for declarations to be possible in if-statements, they <b>must</b> be enclosed in blocks! Else, the declaration would "leak" out of the scope.

A possible question is: what is the purpose of opening an if-statement only to declare a name?

In [ ]:
if( int condition )
    
     int newval = condition + 1; // This is not possible

// The correct way of doing this is
if( int newcondition ){
    
    int declaration = newcondition;
}

### Section 9.4.2 - <span style="color: blue;">switch</span> Statements

Expressions is <b>case</b> labels must be of integral or enumeration type, and it is not possible to repeat values in the former. E.g.:

In [ ]:
int value{ };

switch( value ){
        
    case 2:
        
        break;
        
    case 2: // not allowed. Repeated value.
        
        break;
}

<b>switch</b> <b>if</b> statements are somewhat interchangeable. This means that either one can be written as the other. But, keep in mind: in some cases, it is preferred to use one insted of the other. For example, <b>switch</b> statements are more explicit when testing for constants, i.e., the semantic behind switch is more appropriate for those cases. Thus, consider the semantic when choosing between both <b>switch</b> and <b>if</b>. Also, in terms of performance, according to https://stackoverflow.com/questions/97987/advantage-of-switch-over-if-else-statement, <b>switch</b> <i>might</i> be faster than <b>if</b> statements.

Cases in <b>switch</b> statements must be explicitly terminated, unless we want to proceed executing cases until the last is reached. Proceeding through different cases is called <b>fall-through</b>. But, if no fall-through is desired, we can get out of <b>switch</b> statements with <b>break</b> or <b>return</b>.

Do not use <b>default</b> cases when handling <b>enumeration</b>! Leave the default out, so as to be able to catch enumerators not used in the statement, i.e., that were left out of the latter.

#### 9.4.2.1 - Declarations In Cases

It is possible to declare in cases, but every declaration <b>MUST</b> be initialized, and the cases where both declaration and initialization are <b>MUST</b> be executed. Not doing so will invoke a compiler error. An exception to this is the declaration of <b>int</b> entities. Since they need not be initialized, the declaration can happen without the latter. BUT, the USE cannot happen withou an initialization, because a compiler error will be happen. Anyways, AVOID handling uninitialized variables.

Also, it seems that, unless a block of code restricts a case, anything declared in it are in the scope of the switch statements, instead of a local scope of the case itself.

In [ ]:
#include<string>
#include<iostream>

// The switch statement in wrong( int test ) will not run. This is so because case 0 might happen
// to be bypassed. If the code goes straight to case 1, shit is going down
/*void wrong( int test ){
    
    switch( test ){
            
        case 0:
            
            int x;
            int y = 3;
            
            std::string s;
            
        case 1:
            
            ++x;
            ++y;
            
            s = "nasty";
    }
}*/

// This, however, will run. This is so because x,y and text are strictly case scope bounded, i.e.,
// their declaration is confined to the block of code, or scope, in which they were declared
void right( int value ){
    
    switch( value ){
            
        case 0:{
            
            int x = 0;
            
            ++x;
            
            std::cout << x;
            }
            
        case 1:{
            
            int y = 3;
            
            ++y;
        }
            
        case 2:{
            
            std::string text{ };
            
            text = "aight";
        }
    }
}

right( 0 );

### 9.4.3 - Declarations In Conditions

We can declare <b>INSIDE</b> conditions. More precisely, we can do the following:

In [ ]:
if( double example = prim( true ) ){
    
    break;
}

The declaration happens at condition opening. Its value is provided and tested within the condition, and its scope extends to the end of the block corresponding to the if-statement, i.e., it cannot be accessed outside the latter. But, notice one thing, if there was an <b>else</b> statement here, <b>example</b> would have been accessible in the latter. Only one entity can be declared and initialized in a condition.

Another advantage of this kind of declaration is code compactness.

## 9.5 Iterations Statements

### 9.5.1 - Range-<span style="color: blue;">for</span> Statements

Only works on sequences of elements, i.e., expressions for which <b>.begin() or begin( sequence )</b> and <b>.end() or end( sequence )</b> are provided, because those functions provide us with the iterators of the sequences.

The compiler, in these loops, first tries to access <b>.begin()</b> or <b>.end()</b> member functions. If none of those were provided or are usable, then an error is thrown. Else, the compiler checks for <b>begin( seq )</b> or <b>end( seq )</b> member functions in the scope where the loop is present. If none are found or are usable, an error is thrown.

For built-in arrays, the compiler uses the initial position and the size, i.e., number of elements + 1 or one after-the-last, as <b>begin()</b> and <b>end()</b>, respectively, in range-for loops. Using the <iterator> header, we can access <b>begin( seq )</b> and <b>end( seq )</b> for built-in arrays and std containers. For custom-designed sequences, we can define our own <b>begin()</b> and <b>end()</b> methods, following, of course, the behavior of the same name, similar std methods. 
    
In range-for loops, the variables being iterated are handled as dereferenced pointers in standard for-loops. I.e.,

In [ ]:
std::vector<int> elements{};

// This
for( int value : elements ){
    
    // manipulate value
}

// Is the same as this
for( auto val = begin( elements ); val != end( elements ); ++val ){
    
    // do something
}

To modify an element of a range-for loop, i.e., the variables in a sequence, one must handle those as references, as in the code below:

In [ ]:
void example( std::vector<int>& elements ){
    
    for( int& values : elements ){
        
        // do something with values
    }
}

Handling as references also make it possible to avoid expensive copies of large elements.

### 9.5.2 - <span style="color: blue;">for</span> Statements

If it not the case that the usage of a for-loop follows the standard behavior, as cited by the author, 

    introduce a loop variable
    test the condition
    update loop variable,

then, consider using a <b>while</b>-statement.

We also have a different form of for-statement, as in the code below.

In [ ]:
#include<string>
#include<iostream>

std::vector<std::string> text{ };

for( std::string words; std::cin >> words; ){
    
    text.push_back( words );
    
    if( words == "stop" ){ // Do not focus on this if-statement. This is here only so as
                           // to make it possible for the program to stop
        break;
    }
}

// The example end is above. From here onwards, we are just helping with the visualization

for( std::vector<std::string>::size_type i = 0; i < text.size( ); ++i ){
    
    std::cout << text[ i ] << "\t";
}

In the code above, <b>std::cin >> words</b> both reads values into <b>words</b> and tests for termination, i.e., behaves as a loop-terminating condition checking. The scope of <b>words</b> is restricted to the body of the for-statement, and it is considered an <b>implicit</b> loop variable. While in <b>while</b>-statements condition checking we could also read values, the variables into which we are reading values must be declared <b>outside</b> the <b>while</b>-statement. This means that its scope extends to the end of the block of code where both it and the <b>while</b>-statement are located.

Now, some weirdnesses of C++. We can do the following:

In [ ]:
// an unending loop
for( ; ; ){    
}

// which is equivalent to
while( true ){
}

Yep, that is it. I do not know what is the purpose of this, but, there it is!

Before we end this section: use <b>auto</b> when the type of the loop variable is not evident.

### 9.5.3 - <span style="color: blue;">while</span> Statements

Stroustrup recommends <b>while</b> over <b>for</b>-statements when no obvious loop variable is available or when the updates comes naturally in the loop body.

### 9.5.4 - <span style="color: blue;">do</span> Statements

<b>Avoid those.</b>

Condition comes after the body of the <b>do</b>-statement. This means that the body of the latter is executed at least <b>once</b> before the condition is checked.

This is asking for trouble.

### 9.5.5 - Loop Exit

Forms of forcing ourselves out of a loop where the terminating condition is either ommited or not satisfied.

<b>break</b> breaks away from the nearest switch or iteration-statement. It forces the program out of a loop as soon as it is called. Notice that a break statement, in nested loops or switch statements, a <b>break</b> will leave only the closest, innermost, loop or switch statement in which it is. E.g.

In [ ]:
for( int beginning = 0; beginning < 10; ++beginning ){
    
    for( int middle = 0; middle < 10; ++middle ){
        
        for( int end = 0; end < 10; ++end ){
            
            break; // This will make the program leave this for-loop and go back to 
                   //  the above for, where int middle is
        }
    }
}

Potentially harmful, not because it is bad by itself, but because one can be led to poorly design its loops and "break" away from them before unwanted behavior, which might not have happened if the loop were correct, starts. 

<b>OPEN QUESTION:</b> Stroustrup says "Unless it warps the logic of a loop( e.g., requires the introduction of an extra variable ), it is usually better to have the complete exit condition as the condition of a <b>while</b>-statement or a <b>for</b>-statement." I wonder, how could a complete exit condition as the condition of a loop-statement( that condition that is located in the parenthesis following the loop keyword ) warp the logic of a loop? May it be that using a complete exit condition as the condition of a loop forces us to introduce a new variable in the loop body from which the complete condition takes value? Maybe it is only "complete" with the introduction of new variables? I do not know exactly what Stroustrup means here.

<b>continue</b> makes the loop skip, from its call point, to the next iteration. This means that every instruction that comes after the call to <b>continue</b> will be ignored. Sometimes it is semantically correct and informative, even if apparently "useless" in a loop body. Use it carefully!

## 9.6 - <span style="color: blue;">goto</span> Statements

The <b>goto</b> statement has the following form:

    <b>goto</b> identifier;
    identifier : statement;
    
Its scope is the function where it is in. Yep, the WHOLE function. But there are some restrictions. Be them:
 
     1. if there is an initializer after the goto, it cannot jump past it
     2. it cannot be used go to into an exception
     
<b>goto</b>, according to Stroustrup, can "sensibly", read reasonably, be used to break out from nested loops or switch statements.

## 9.7 - Comments And Indentation

According to Stroustrup, if something can semantically be stated in the language itself( think about self-documented code ), then it should, and should not be mentioned in a comment.

Good comments should inform what the code is supposed to do( almost literal transcription of the text ). Higher levels of abstraction are preferred when doing as aforementioned.

Stroustrup expresses his preferences for comments in page 239 of this book. Taken from the text( Stroustrup, 2013 ), some of them are:

    • A comment for each source file stating what the declarations in it have in common, refer-
    ences to manuals, the name of the programmer, general hints for maintenance, etc.
    • A comment for each class, template, and namespace
    • A comment for each nontrivial function stating its purpose, the algorithm used (unless it is
    obvious), and maybe something about the assumptions it makes about its environment
    • A comment for each global and namespace variable and constant
    • A few comments where the code is nonobvious and/or nonportable

<b>/* */</b> do not nest, i.e., we cannot do

In [ ]:
/*

    /*
    */ // This will close commenting

*/ // And this will make no sense at all, and will return an error message from the linter

# Chapter 10 - Expressions

This chapter is less theoretical and very code oriented. So, it might be better to just read it, there are few things here to actually take notes about.

## 10.1 - Introduction

## 10.2 - A Desk Calculator

### 10.2.1 - The Parser

C++ provides assignment operators for binary operators( almost exactly taken from the text ), i.e., 

    + - * / % & | ^ << >>
    
can be combined with <b>=</b>, resulting in

    += -= #= /= %= &= |= ^= <<= >>=
    
where **&**, <b>|</b> and <b>^</b> are the bitwise logical operators and, or, and XOR, and **<<** and **>>** are left and right shift operators.

### 10.2.2 - Input

### 10.2.3 - Low-Level Input

Things like <b>isspace()</b>, <b>isdigit()</b>, <b>isalpha()</b>, etc.

### 10.2.4 - Error Handling

### 10.2.5 - The Driver

### 10.2.6 - Headers

### 10.2.7 - Command- Line Arguments

Remember those two arguments passed to <b>main()</b> that we never knew the purpose of or what they did? Yes? Well, the first argument, <b>int argc</b>, represents the number of arguments that will be passed to <b>main()</b>, and <b>char* argv[ ]</b> represents an array of arguments passed to <b>main()</b>. The type of the elements in this array of arguments is

<b>char* [ argc+ 1 ]</b>

i.e., a pointer to an array of chars, with size <b>argc + 1</b>, and with <b>argv[ 0 ]</b> equal to the name of the program, i.e., <b>argc</b> at least 1. Also, <b>argv[ argc ] == 0</b>, i.e., zero-terminated. Too se more about that, refer to https://stackoverflow.com/questions/4207134/what-is-the-proper-declaration-of-main

### 10.2.8 - A Note On Style

## 10.3 - Operator Summary

Some quick things to have on mind before proceeding:

• <i>name</i> are identifiers, operator's names, or the name of templates specializations
• <i>Lvalue</i> are expressions representing modifiable objects
• <i>Lambda-declarator</i> are lists of parameters, possibly followed by <b>mutable</b>, <b>noexcept</b> and a <i>return trailing type</i>
• <i>Capture-list</i> is a list that allows scope handling/extending of variables to lambdas
• <i>stmt-list</i> is a list of statements that goes into a lambda body

Aside from that, there are a LOT of tables summarizing operators in this section, going from p.255 to p.257. Use as reference.